In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from cedenar_anomalies.utils.paths import (
    data_processed_dir,
    data_raw_dir,
    data_interim_dir
)

In [2]:
# !pip uninstall matplotlib
# !pip cache purge #Clean cache
# !pip install matplotlib

In [3]:
os.getcwd()

'/Users/frandak2/Documents/repos/cedenar_anomalies/notebooks'

In [4]:
pd.set_option("display.max_rows", None)  # Mostrar todas las filas
pd.set_option("display.max_columns", None)  # Mostrar todas las columnas

In [5]:
import pandas as pd
ruta_archivo = data_raw_dir("anomalias 2022 23 y 24.xlsx")
df = pd.read_excel(ruta_archivo)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34683 entries, 0 to 34682
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Zona         34683 non-null  object 
 1   Contrato     34683 non-null  object 
 2   Orden        34683 non-null  int64  
 3   Usuario      34683 non-null  int64  
 4   Estado       34683 non-null  object 
 5   Ejecucion    34683 non-null  object 
 6   Revision     29324 non-null  object 
 7   Codigo       30019 non-null  object 
 8   Anomalia     34683 non-null  object 
 9   N. Unidad    34683 non-null  int64  
 10  Unidad       34682 non-null  object 
 11  N. Causa     34683 non-null  int64  
 12  Causa        34682 non-null  object 
 13  U. Genera    34683 non-null  object 
 14  Descripcion  34683 non-null  object 
 15  Motivo       33671 non-null  object 
 16  Cuadrilla    34683 non-null  object 
 17  kWh Rec      23088 non-null  float64
 18  Factor       775 non-null    float64
dtypes: f

In [6]:
df["Zona"].value_counts()

Zona
PA    12045
NO     8908
CE     7381
SU     3764
OC     2585
Name: count, dtype: int64

In [7]:
# Filtrar por la variable Zona igual a "PA"
df_pa = df[df["Zona"] == "PA"].copy()

In [8]:
# Crear un nuevo dataset con las columnas especificadas
columnas_deseadas = [
    "Orden",
    "Usuario",
    "Ejecucion",
    "Estado",
    "Codigo",
    "Anomalia",
    "Causa",
    "Descripcion",
    "Motivo",
    "kWh Rec",
    "Factor",
]
df_nuevo = df_pa[columnas_deseadas].copy()

In [9]:
# Identificar columnas problemáticas
columnas_problema = df_nuevo.select_dtypes(include=["object"]).columns

# Convertir columnas problemáticas a cadenas de texto
for col in columnas_problema:
    df_nuevo[col] = df_nuevo[col].astype(str)

In [10]:
import pandas as pd

# Suponiendo que df_nuevo ya está definido

# Crear una copia del DataFrame para evitar modificar el original
df_nuevo_copia = df_nuevo.copy()


# Función para expandir los registros con múltiples códigos
def expandir_codigos(row):
    codigos = str(row["Codigo"]).split("/")
    nuevas_filas = []
    for codigo in codigos:
        nueva_fila = row.copy()
        nueva_fila["Codigo"] = (
            codigo.strip()
        )  # Eliminar espacios en blanco alrededor del código
        nuevas_filas.append(nueva_fila)
    return pd.DataFrame(nuevas_filas)


# Aplicar la función a las filas con "/" en la columna "Codigo"
filas_a_expandir = df_nuevo_copia[df_nuevo_copia["Codigo"].str.contains("/")]
nuevos_registros = pd.concat(
    filas_a_expandir.apply(expandir_codigos, axis=1).tolist(), ignore_index=True
)

# Eliminar los registros originales con "/" en la columna "Codigo"
df_nuevo_copia = df_nuevo_copia[~df_nuevo_copia["Codigo"].str.contains("/")]

# Concatenar los nuevos registros con el DataFrame original
df_nuevo_copia = pd.concat([df_nuevo_copia, nuevos_registros], ignore_index=True)

# Guardar el DataFrame modificado en un archivo Excel (opcional)
# df_nuevo_copia.to_csv(data_interim_dir("df_nuevo_codigos_separados.csv"), index=False)

In [11]:
columnas_deseadas = [
    "Orden",
    "Usuario",
    "Ejecucion",
    "Codigo",
    "Descripcion",
    "Motivo",
    "kWh Rec",
    "Factor",
]
df_anom_sep = df_nuevo_copia[columnas_deseadas].copy()

In [12]:
df_anom_sep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37767 entries, 0 to 37766
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Orden        37767 non-null  int64  
 1   Usuario      37767 non-null  int64  
 2   Ejecucion    37767 non-null  object 
 3   Codigo       37767 non-null  object 
 4   Descripcion  37767 non-null  object 
 5   Motivo       37767 non-null  object 
 6   kWh Rec      31475 non-null  float64
 7   Factor       11 non-null     float64
dtypes: float64(2), int64(2), object(4)
memory usage: 2.3+ MB


In [13]:
df_anom_sep.head(2)

,Orden,Usuario,Ejecucion,Codigo,Descripcion,Motivo,kWh Rec,Factor
0,11168065,544326,2021-03-27 08:04:44-05,30004,Anomalia verificada y aprobada,Otras Anomalias,NaN,NaN
1,11232649,1142265,2021-04-09 10:56:34-05,30004,Posible anomalia,nan,NaN,NaN


In [7]:
ruta_archivo = data_raw_dir("items anomalia ponderado.xlsx")
df_ponderado = pd.read_excel(ruta_archivo)

In [8]:
df_ponderado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          63 non-null     int64 
 1   Sigla       63 non-null     object
 2   Movil       63 non-null     int64 
 3   Item        62 non-null     object
 4   Creado      63 non-null     object
 5   Nombre      63 non-null     object
 6   Activo      62 non-null     object
 7   puntaje     63 non-null     int64 
 8   evaluacion  63 non-null     int64 
dtypes: int64(4), object(5)
memory usage: 4.6+ KB


In [16]:
df_anom_sep_copia = df_anom_sep.copy()

# Realizar la combinación (merge) de los DataFrames
df_anom_sep_copia = pd.merge(
    df_anom_sep_copia, df_ponderado, left_on="Codigo", right_on="Item", how="left"
)

# Seleccionar solo las columnas deseadas
columnas_deseadas = list(df_anom_sep.columns) + ["id", "Nombre", "puntaje", "evaluacion"]
df_anom_sep_copia = df_anom_sep_copia[columnas_deseadas]

# Guardar el DataFrame modificado en un archivo Excel (opcional)
# df_anom_sep_copia.to_csv(data_interim_dir("df_anom_sep_extendido.csv"), index=False)

In [17]:
df_anom_sep_copia.head(3)

,Orden,Usuario,Ejecucion,Codigo,Descripcion,Motivo,kWh Rec,Factor,id,Nombre,puntaje,evaluacion
0,11168065,544326,2021-03-27 08:04:44-05,30004,Anomalia verificada y aprobada,Otras Anomalias,NaN,NaN,40.0,OTRAS ANOMALÍAS REINCIDENTE PARE,1.0,1.0
1,11232649,1142265,2021-04-09 10:56:34-05,30004,Posible anomalia,nan,NaN,NaN,40.0,OTRAS ANOMALÍAS REINCIDENTE PARE,1.0,1.0
2,11353009,543180,2021-04-06 09:30:10-05,30004,Anomalia verificada y aprobada,Otras Anomalias,NaN,NaN,40.0,OTRAS ANOMALÍAS REINCIDENTE PARE,1.0,1.0


In [18]:
df_anom_sep_copia["evaluacion"].value_counts()

evaluacion
3.0    77584
1.0    41973
2.0    41476
Name: count, dtype: int64

In [19]:
df_anom_sep_copia.columns

Index(['Orden', 'Usuario', 'Ejecucion', 'Codigo', 'Descripcion', 'Motivo',
       'kWh Rec', 'Factor', 'id', 'Nombre', 'puntaje', 'evaluacion'],
      dtype='object')

In [9]:
ruta_archivo = data_raw_dir("cedenar_data.xlsx")
usu_pacifico = pd.read_excel(ruta_archivo)
df_usuarios = usu_pacifico[usu_pacifico.ZONA == "PACIFICO"].copy()

In [10]:
usu_pacifico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 622450 entries, 0 to 622449
Data columns (total 58 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   PRODUCT_ID                  622450 non-null  int64  
 1   PRODUCTO                    622450 non-null  int64  
 2   CONTRATO                    622450 non-null  int64  
 3   CLIENTE                     622450 non-null  int64  
 4   NOMBRE_CLIENTE              622450 non-null  object 
 5   APELLIDO_CLIENTE            111432 non-null  object 
 6   ZONA                        622286 non-null  object 
 7   SECCIONAL                   622286 non-null  object 
 8   DIRECCION_PRODUCTO          622450 non-null  object 
 9   MUNICIPIO_PRODUCTO          622450 non-null  object 
 10  VEREDA_CENTRO_POBLADO       600815 non-null  object 
 11  BARRIO_PRODUCTO             600815 non-null  object 
 12  SECTOR_OPERATIVO_PRODUCTO   622450 non-null  object 
 13  COD_CICLO_CONS

In [21]:
df_usuarios.head(2)

,PRODUCT_ID,PRODUCTO,CONTRATO,CLIENTE,NOMBRE_CLIENTE,APELLIDO_CLIENTE,ZONA,SECCIONAL,DIRECCION_PRODUCTO,MUNICIPIO_PRODUCTO,VEREDA_CENTRO_POBLADO,BARRIO_PRODUCTO,SECTOR_OPERATIVO_PRODUCTO,COD_CICLO_CONSUMO,CICLO_CONSUMO,COD_CICLO_FACTURACION,CICLO_FACTURACION,RUTA,DES_RUTA,ORDEN_PREDIO,AREA,FECHA_INSTALACION_PRODUCTO,ESTADO_DE_CORTE,ESTADO_PRODUCTO,FECHA_RETIRO_PRODUCTO,SERVICIO_PROVISIONAL,PRODUCTO_ACTIVO,CATEGORIA,SUB_CATEGORIA,ZONA_TARIFARIA,PLAN_COMERCIAL,METOD_VARCON,TRAFO_OPEN,FECHA_INGRESO_A_SISTEMA,FECHA_DE_CONEXION,CARTERA_EN_MORA,CARTERA_TOTAL,CARGA_INSTALADA,MODO_ADQUISICION,MEDIDOR,COMPONENTE,CODIGO_MEDIDOR,FECHA_INSTALACION,ESTADO_MEDIDOR,COMPONENT_TYPE_ID,CLASE_SERVICIO,TIPO_MEDIDOR,GAMA,TIPO_ITEM,FASES,TRAFO_SPARD,KVA,CIRCUITO,NIVEL,LATI_USU,LONG_USU,LATI_TRAFO,LONG_TRAFO
146,597102,597102,597102,100706,MARCIAL VICTORIANO_ LOPEZ NARVAEZ,NaN,PACIFICO,PACIFICO,EL ESFURZO I CS 347 DIV 1,TUMACO,SAN ANDRES DE TUMACO DISTRITO ESPECIAL INDUS...,EL ESFUERZO,114-TUMACO URBANO,4,0Y-TARIFA PARA CLAUSURA PACIFI,4,0Y-TARIFA PARA CLAUSURA PACIFI,5.0,2182-RUTA 2182 DE TUMACO,6058.0,Urbano,2000-10-10,92-Retiro definitivo,3-Retirado,2020-07-06,N,N,1-RESIDENCIAL,1-ESTRATO 1,-1-POR DEFECTO,16-RESIDENCIAL - DIRECTO,5013-PRODUCTOS DIRECTOS,TM30006,2000-10-10,2000-10-10,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,1184921,1184921,1184921,100802,ZAIDA JACKELINE,VALVERDE QUIÑONES,PACIFICO,PACIFICO,CS 53 A PRADOMAR DIV 1,TUMACO,SAN ANDRES DE TUMACO DISTRITO ESPECIAL INDUS...,PRADOMAR,114-TUMACO URBANO,54,0G-MEDIDA CENTRALIZADA TUMACO,54,0G-MEDIDA CENTRALIZADA TUMACO,4040.0,7510 - TUMACO VIADUCTO CONVENCIONAL MORRITO TR...,35560.0,Urbano,2016-02-29,1-Conexión,1-Activo,4732-12-31,N,Y,1-RESIDENCIAL,1-ESTRATO 1,-1-POR DEFECTO,31-PLAN RESIDENCIAL TELEMEDIDO,5004-CLIENTES TELEMEDIDOS,60TA030352,2016-02-29,2016-02-29,0.0,47090.0,1.8,C - comodato,MEC-54186,1605850.0,301790.0,2016-02-29,21-Pendiente de Retiro,18.0,3257-MEDIDOR AMI,NaN,NaN,NaN,NaN,60TA030352,45.0,60TM03,1.0,1.819331,-78.749592,1.819363,-78.749388


In [22]:
columnas_deseadas = [
    "PRODUCTO",
    "AREA",
    "PLAN_COMERCIAL",
    "TRAFO_OPEN",
    "FASES",
    "KVA",
    "LATI_USU",
    "LONG_USU",
]
df_usuarios_f = df_usuarios[columnas_deseadas].copy()

In [23]:
df_usuarios_f.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86423 entries, 146 to 622449
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PRODUCTO        86423 non-null  int64  
 1   AREA            86409 non-null  object 
 2   PLAN_COMERCIAL  86423 non-null  object 
 3   TRAFO_OPEN      84291 non-null  object 
 4   FASES           67434 non-null  float64
 5   KVA             73318 non-null  float64
 6   LATI_USU        73318 non-null  float64
 7   LONG_USU        73318 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 5.9+ MB


In [24]:
usuarios_unicos = pd.DataFrame(df_anom_sep_copia["Usuario"].unique(), columns=["Usuario"])
# usuarios_unicos.to_excel("usuarios_unicos_anomal.xlsx", index=False)

print("Valores únicos de 'Usuario' guardados en usuarios_unicos_anomal.xlsx")

# 2. Crear un nuevo dataset comparando y agregando columnas
df_nuevo_extendido = pd.merge(
    df_anom_sep_copia, df_usuarios_f, left_on="Usuario", right_on="PRODUCTO", how="left"
)

# Seleccionar solo las columnas deseadas
columnas_deseadas = [
    "Orden",
    "Usuario",
    "Ejecucion",
    "Codigo",
    "Descripcion",
    "Motivo",
    "kWh Rec",
    "Factor",
    "id",
    "Nombre",
    "Factor",
    "AREA",
    "PLAN_COMERCIAL",
    "TRAFO_OPEN",
    "FASES",
    "KVA",
    "LATI_USU",
    "LONG_USU",
    "puntaje",
    "evaluacion",
]
df_nuevo_extendido = df_nuevo_extendido[columnas_deseadas]

# Mostrar las primeras filas del nuevo dataset extendido
print("\nNuevo dataset extendido (df_nuevo_extendido):")

# Guardar el nuevo dataset extendido en un archivo Excel (opcional)
# df_nuevo_extendido.to_excel("df_nuevo_extendido.xlsx", index=False)

Valores únicos de 'Usuario' guardados en usuarios_unicos_anomal.xlsx

Nuevo dataset extendido (df_nuevo_extendido):


In [25]:
df_nuevo_extendido.head(2)

,Orden,Usuario,Ejecucion,Codigo,Descripcion,Motivo,kWh Rec,Factor,id,Nombre,Factor,AREA,PLAN_COMERCIAL,TRAFO_OPEN,FASES,KVA,LATI_USU,LONG_USU,puntaje,evaluacion
0,11168065,544326,2021-03-27 08:04:44-05,30004,Anomalia verificada y aprobada,Otras Anomalias,NaN,NaN,40.0,OTRAS ANOMALÍAS REINCIDENTE PARE,NaN,Urbano,2-PLAN RESIDENCIAL,05TN000137,2.0,45.0,1.672157,-78.142107,1.0,1.0
1,11232649,1142265,2021-04-09 10:56:34-05,30004,Posible anomalia,nan,NaN,NaN,40.0,OTRAS ANOMALÍAS REINCIDENTE PARE,NaN,Urbano,2-PLAN RESIDENCIAL,05TN000114,1.0,45.0,1.673809,-78.140233,1.0,1.0


In [26]:
# df_nuevo_extendido.to_csv(data_interim_dir("df_nuevo_extendido.csv"), index=False)

In [27]:
# Crear una copia del DataFrame original
df_label_encoded_cluster = df_nuevo_extendido.copy()

# Lista para almacenar los mapeos de niveles originales y sus códigos
category_mappings = {}

# Columnas a codificar
columns_to_encode = ["id", "AREA", "PLAN_COMERCIAL"]  # Ajusta a tus columnas

# Aplicar Label Encoding a cada columna
for col in columns_to_encode:
    # Convertir a tipo categoría
    df_label_encoded_cluster[col] = df_label_encoded_cluster[col].astype("category")

    # Guardar el mapeo de niveles
    category_mappings[col] = dict(enumerate(df_label_encoded_cluster[col].cat.categories))

    # Reemplazar los valores por sus códigos numéricos
    df_label_encoded_cluster[col] = df_label_encoded_cluster[col].cat.codes

In [28]:
import json

# Guardar en archivo JSON
with open("category_mappings_anomaly_ext.json", "w") as f:
    json.dump(category_mappings, f)

# Cargar desde archivo JSON
with open("category_mappings_anomaly_ext.json", "r") as f:
    loaded_mappings = json.load(f)

In [29]:
df_label_encoded_cluster.to_csv(data_interim_dir("df_anomaly_limpio_train.csv"), index=False)

In [30]:
columnas_deseadas = [
    "Usuario",
    "id",
    "AREA",
    "PLAN_COMERCIAL",
    "LATI_USU",
    "LONG_USU",
    "puntaje",
    "evaluacion",
]
df_label_encoded_extendido = df_label_encoded_cluster[columnas_deseadas]

In [31]:
df_label_encoded_extendido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161685 entries, 0 to 161684
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Usuario         161685 non-null  int64  
 1   id              161685 non-null  int8   
 2   AREA            161685 non-null  int8   
 3   PLAN_COMERCIAL  161685 non-null  int8   
 4   LATI_USU        161328 non-null  float64
 5   LONG_USU        161328 non-null  float64
 6   puntaje         161033 non-null  float64
 7   evaluacion      161033 non-null  float64
dtypes: float64(4), int64(1), int8(3)
memory usage: 6.6 MB


In [32]:
df_label_encoded_extendido.head()

,Usuario,id,AREA,PLAN_COMERCIAL,LATI_USU,LONG_USU,puntaje,evaluacion
0,544326,22,2,4,1.672157,-78.142107,1.0,1.0
1,1142265,22,2,4,1.673809,-78.140233,1.0,1.0
2,543180,22,2,4,1.671898,-78.140568,1.0,1.0
3,543302,22,2,4,1.672003,-78.141612,1.0,1.0
4,544763,22,2,4,1.675486,-78.137020,1.0,1.0


In [33]:
df_label_encoded_extendido.to_csv(data_interim_dir("df_anomaly_encoded_limpio_train.csv"), index=False)

# Resumen del notebook

In [85]:
df_anomalies = pd.read_excel(data_raw_dir("anomalias 2022 23 y 24.xlsx"))
df_anomalies_pa = df_anomalies[df_anomalies["Zona"] == "PA"].copy()
df_ponderado = pd.read_excel(data_raw_dir("items anomalia ponderado.xlsx"))
df_usuarios = pd.read_excel(data_raw_dir("cedenar_data.xlsx"))
df_usuarios_pa = df_usuarios[usu_pacifico.ZONA == "PACIFICO"].copy()

In [67]:
cols_anomalies = ["Orden", "Usuario","Ejecucion", "Codigo","Descripcion", "Motivo", "kWh Rec", "Factor", "Zona"]
df_anomalies_pa = df_anomalies_pa[cols_anomalies].copy()
df_expanded = df_anomalies.assign(Codigo=df_anomalies['Codigo'].astype(str).str.split("/")).explode("Codigo")
df_expanded['Codigo'] = df_expanded['Codigo'].str.strip()

df_merged = pd.merge(df_expanded, df_ponderado[["Item",'id','Nombre','puntaje', 'evaluacion']].copy(), how="left", left_on="Codigo", right_on="Item")
df_merged.drop(columns=['Item'], inplace=True)
columnas_deseadas = ["PRODUCTO", "AREA", "PLAN_COMERCIAL","TRAFO_OPEN", "FASES", "KVA", "LATI_USU", "LONG_USU"]
df_usuarios = df_usuarios[columnas_deseadas].copy()

# 2. Crear un nuevo dataset comparando y agregando columnas
df_nuevo_extendido = pd.merge(df_merged, df_usuarios, left_on='Usuario', right_on='PRODUCTO', how='left')

# Seleccionar solo las columnas deseadas
columnas_deseadas = ['Orden', 'Usuario', "Ejecucion", 'Codigo', 'Descripcion', 'Motivo', 'kWh Rec','Factor', 'id','Nombre','Factor','AREA', 'PLAN_COMERCIAL', 'TRAFO_OPEN', 'FASES', 'KVA', 'LATI_USU', 'LONG_USU','puntaje', 'evaluacion', "Zona"]
df_nuevo_extendido = df_nuevo_extendido[columnas_deseadas]

In [68]:
df_expanded.head(2)

,Zona,Contrato,Orden,Usuario,Estado,Ejecucion,Revision,Codigo,Anomalia,N. Unidad,Unidad,N. Causa,Causa,U. Genera,Descripcion,Motivo,Cuadrilla,kWh Rec,Factor
0,CE,DES 100.2018,10141926,1154748,Pendiente,2021-02-22 12:54:26-05,NaN,2,ACOMETIDA O DERIVACIÓN NO AUTORIZADA HACIA CAR...,0,GES. ENERGETICA,0,CAUSA NO DETERMINADA,LPATINO123,Posible anomalia,NaN,ZTY143,NaN,NaN
0,CE,DES 100.2018,10141926,1154748,Pendiente,2021-02-22 12:54:26-05,NaN,1,ACOMETIDA O DERIVACIÓN NO AUTORIZADA HACIA CAR...,0,GES. ENERGETICA,0,CAUSA NO DETERMINADA,LPATINO123,Posible anomalia,NaN,ZTY143,NaN,NaN


In [69]:
df_ponderado.head(2)

,id,Sigla,Movil,Item,Creado,Nombre,Activo,puntaje,evaluacion
0,6,LD,143,82,2018-09-18,LINEA DIRECTA,SI,5,3
1,23,AI,144,38,2018-09-18,ACOMETIDA INTERVENID,SI,5,3


In [70]:
df_merged = pd.merge(df_expanded, df_ponderado[["Item",'id','Nombre','puntaje', 'evaluacion']].copy(), how="left", left_on="Codigo", right_on="Item")

df_merged.drop(columns=['Item'], inplace=True)

In [71]:
df_merged.head(2)

,Zona,Contrato,Orden,Usuario,Estado,Ejecucion,Revision,Codigo,Anomalia,N. Unidad,Unidad,N. Causa,Causa,U. Genera,Descripcion,Motivo,Cuadrilla,kWh Rec,Factor,id,Nombre,puntaje,evaluacion
0,CE,DES 100.2018,10141926,1154748,Pendiente,2021-02-22 12:54:26-05,NaN,2,ACOMETIDA O DERIVACIÓN NO AUTORIZADA HACIA CAR...,0,GES. ENERGETICA,0,CAUSA NO DETERMINADA,LPATINO123,Posible anomalia,NaN,ZTY143,NaN,NaN,73.0,ACOMETIDA O DERIVACIÓN NO AUTORIZADA HA,5.0,3.0
1,CE,DES 100.2018,10141926,1154748,Pendiente,2021-02-22 12:54:26-05,NaN,1,ACOMETIDA O DERIVACIÓN NO AUTORIZADA HACIA CAR...,0,GES. ENERGETICA,0,CAUSA NO DETERMINADA,LPATINO123,Posible anomalia,NaN,ZTY143,NaN,NaN,69.0,ACOMETIDA O DERIVACIÓN NO AUTORIZADA HA,5.0,3.0


In [72]:
df_merged[df_merged.Orden == 11168065]

,Zona,Contrato,Orden,Usuario,Estado,Ejecucion,Revision,Codigo,Anomalia,N. Unidad,Unidad,N. Causa,Causa,U. Genera,Descripcion,Motivo,Cuadrilla,kWh Rec,Factor,id,Nombre,puntaje,evaluacion
22,PA,PIE 333.2021,11168065,544326,Aprobada,2021-03-27 08:04:44-05,2022-02-26 04:24:33.66772-05,30004,OTRAS ANOMALÍAS REINCIDENTE PARE,0,CONTROL PERDIDAS,0,CAUSA NO DETERMINADA,GSANDOVAL,Anomalia verificada y aprobada,Otras Anomalias,ZTY971,NaN,NaN,40.0,OTRAS ANOMALÍAS REINCIDENTE PARE,1.0,1.0


In [73]:
columnas_deseadas = ["PRODUCTO", "AREA", "PLAN_COMERCIAL","TRAFO_OPEN", "FASES", "KVA", "LATI_USU", "LONG_USU"]
df_usuarios = df_usuarios[columnas_deseadas].copy()

# 2. Crear un nuevo dataset comparando y agregando columnas
df_nuevo_extendido = pd.merge(df_merged, df_usuarios, left_on='Usuario', right_on='PRODUCTO', how='left')

# Seleccionar solo las columnas deseadas
columnas_deseadas = ['Orden', 'Usuario', "Ejecucion", 'Codigo', 'Descripcion', 'Motivo', 'kWh Rec','Factor', 'id','Nombre','Factor','AREA', 'PLAN_COMERCIAL', 'TRAFO_OPEN', 'FASES', 'KVA', 'LATI_USU', 'LONG_USU','puntaje', 'evaluacion', "Zona"]
df_nuevo_extendido = df_nuevo_extendido[columnas_deseadas]
df_nuevo_extendido.head(2)

,Orden,Usuario,Ejecucion,Codigo,Descripcion,Motivo,kWh Rec,Factor,id,Nombre,Factor,AREA,PLAN_COMERCIAL,TRAFO_OPEN,FASES,KVA,LATI_USU,LONG_USU,puntaje,evaluacion,ZONA,Zona
0,10141926,1154748,2021-02-22 12:54:26-05,2,Posible anomalia,NaN,NaN,NaN,73.0,ACOMETIDA O DERIVACIÓN NO AUTORIZADA HA,NaN,Rural,17-NO RESIDENCIAL - DIRECTO,15TA061951,3.0,15.0,1.396476,-77.288275,5.0,3.0,CENTRO,CE
1,10141926,1154748,2021-02-22 12:54:26-05,1,Posible anomalia,NaN,NaN,NaN,69.0,ACOMETIDA O DERIVACIÓN NO AUTORIZADA HA,NaN,Rural,17-NO RESIDENCIAL - DIRECTO,15TA061951,3.0,15.0,1.396476,-77.288275,5.0,3.0,CENTRO,CE


In [74]:
df_nuevo_extendido[df_nuevo_extendido.Orden == 11168065]

,Orden,Usuario,Ejecucion,Codigo,Descripcion,Motivo,kWh Rec,Factor,id,Nombre,Factor,AREA,PLAN_COMERCIAL,TRAFO_OPEN,FASES,KVA,LATI_USU,LONG_USU,puntaje,evaluacion,ZONA,Zona
22,11168065,544326,2021-03-27 08:04:44-05,30004,Anomalia verificada y aprobada,Otras Anomalias,NaN,NaN,40.0,OTRAS ANOMALÍAS REINCIDENTE PARE,NaN,Urbano,2-PLAN RESIDENCIAL,05TN000137,2.0,45.0,1.672157,-78.142107,1.0,1.0,PACIFICO,PA


In [90]:
def merge_and_clean(df_anomalies, df_users, df_ponderado ) -> pd.DataFrame:
    cols_anomalies = ["Orden", "Usuario","Ejecucion", "Codigo","Descripcion", "Motivo", "kWh Rec", "Factor", "Zona"]
    df_anomalies = self.df_anomalies[cols_anomalies].copy()

    df_expanded = df_anomalies.assign(Codigo=df_anomalies['Codigo'].astype(str).str.split("/")).explode("Codigo")
    df_expanded['Codigo'] = df_expanded['Codigo'].str.strip()

    df_merge_anom_pond = pd.merge(df_expanded, self.df_ponderado[["Item",'id','Nombre','puntaje', 'evaluacion']].copy(), how="left", left_on="Codigo", right_on="Item")
    df_merge_anom_pond.drop(columns=['Item'], inplace=True)

    columnas_deseadas = ["PRODUCTO", "AREA", "PLAN_COMERCIAL","TRAFO_OPEN", "FASES", "KVA", "LATI_USU", "LONG_USU"]
    df_users = self.df_users[columnas_deseadas].copy()

    df_merge_anom_pond_us = pd.merge(df_merge_anom_pond, df_users, left_on='Usuario', right_on='PRODUCTO', how='left')

    columnas_deseadas = ['Usuario',  'id', 'AREA', 'PLAN_COMERCIAL', 'LATI_USU', 'LONG_USU','puntaje', 'evaluacion' "Zona"]
    return df_merge_anom_pond_us[columnas_deseadas].copy()


In [91]:
df = merge_and_clean(df_anomalies, df_usuarios, df_ponderado)

In [92]:
df[df.Orden == 11168065]

,Orden,Usuario,Ejecucion,Codigo,Descripcion,Motivo,kWh Rec,Factor,id,Nombre,Factor,AREA,PLAN_COMERCIAL,TRAFO_OPEN,FASES,KVA,LATI_USU,LONG_USU,puntaje,evaluacion,Zona
22,11168065,544326,2021-03-27 08:04:44-05,30004,Anomalia verificada y aprobada,Otras Anomalias,NaN,NaN,40.0,OTRAS ANOMALÍAS REINCIDENTE PARE,NaN,Urbano,2-PLAN RESIDENCIAL,05TN000137,2.0,45.0,1.672157,-78.142107,1.0,1.0,PA


In [93]:
df[df.Zona == 'PA'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 161685 entries, 13 to 363220
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Orden           161685 non-null  int64  
 1   Usuario         161685 non-null  int64  
 2   Ejecucion       161685 non-null  object 
 3   Codigo          161685 non-null  object 
 4   Descripcion     161685 non-null  object 
 5   Motivo          156086 non-null  object 
 6   kWh Rec         137965 non-null  float64
 7   Factor          41 non-null      float64
 8   id              161033 non-null  float64
 9   Nombre          161033 non-null  object 
 10  Factor          41 non-null      float64
 11  AREA            161663 non-null  object 
 12  PLAN_COMERCIAL  161663 non-null  object 
 13  TRAFO_OPEN      161537 non-null  object 
 14  FASES           160060 non-null  float64
 15  KVA             161575 non-null  float64
 16  LATI_USU        161575 non-null  float64
 17  LONG_USU      

In [9]:
df = pd.read_csv(data_processed_dir("dataset_train_clean.csv"))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363222 entries, 0 to 363221
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Orden           363222 non-null  int64  
 1   Usuario         363222 non-null  int64  
 2   Ejecucion       363222 non-null  object 
 3   Codigo          358558 non-null  object 
 4   Descripcion     363222 non-null  object 
 5   Motivo          354392 non-null  object 
 6   kWh Rec         304578 non-null  float64
 7   Factor          3426 non-null    float64
 8   id              351794 non-null  float64
 9   Nombre          351794 non-null  object 
 10  Factor.1        3426 non-null    float64
 11  AREA            363200 non-null  object 
 12  PLAN_COMERCIAL  363200 non-null  object 
 13  TRAFO_OPEN      363027 non-null  object 
 14  FASES           359828 non-null  float64
 15  KVA             362675 non-null  float64
 16  LATI_USU        362675 non-null  float64
 17  LONG_USU  

In [10]:
columnas_deseadas = [
    "Usuario",
    "id",
    "AREA",
    "PLAN_COMERCIAL",
    "LATI_USU",
    "LONG_USU",
    "puntaje",
    "evaluacion",
]
df[columnas_deseadas].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363222 entries, 0 to 363221
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Usuario         363222 non-null  int64  
 1   id              351794 non-null  float64
 2   AREA            363200 non-null  object 
 3   PLAN_COMERCIAL  363200 non-null  object 
 4   LATI_USU        362675 non-null  float64
 5   LONG_USU        362675 non-null  float64
 6   puntaje         351794 non-null  float64
 7   evaluacion      351794 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 22.2+ MB
